In [2]:
import requests
import json
from base64 import b64encode
import tkinter as tk
import random
from bs4 import BeautifulSoup

In [24]:
client_id = "e291792abd4b4a1db646f561c823beb9"
client_secret = "26e6df1ae3614825940739dd03c31feb"

auth_url = 'https://accounts.spotify.com/api/token'
auth_header = 'Basic ' + b64encode((client_id + ':' + client_secret).encode()).decode()
url = "https://api.spotify.com/v1/search"

auth_data = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post(auth_url, headers={'Authorization': auth_header}, data=auth_data)

if auth_response.status_code == 200:
    token = auth_response.json().get('access_token')
    print('Token:', token)
else:
    print('Error:', auth_response.status_code)
    print(auth_response.text)

Token: BQBR3sImEK1NtbRZtfkc9VJmdIHZmaLYGEQOGexmD8QKOULSf0K0N-0i3QtRa_udUbnKxi1UqDvjnx86OBHdDY3TqQLccCbfbYsQ8nAy7GQdOV4c-eSC_rxdMfFEOVladAMhIw6Y7uY


In [4]:
token = "BQDLS51gIeE-tdAjx3S78yTN_8nus2WYiwXYS_tQE6ktSSIp2A-DaB8pwY-7Gm6iRnXOfkfGM3LmkB6LU-j07Vm3TRvAYmUUgVdAie09YVmUqwqrUtevOQAfvuBkF74RJQGnnECTNbs"

#artist_name = "Taylor Swift"

def get_spotify_id(artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1  
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        results = response.json()
        if results["artists"]["items"]:
            artist = results["artists"]["items"][0]
            id = artist["id"]
            return id
        else:
            return "No artist found."
    else:
        return f"Error: {response.status_code} - {response.text}"

In [23]:
API_KEY = "OdOTpaxGkQyamSaW2EkeanimV3vOKKdmyBTwQHGHxErCN97F3c1Z4kVR8UJWgM85"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

#added for reference
def q2():

    score = 0

    #question 1
    response_2 = input("Enter an artist's name: ")
    question_1 = get_lyrics(response_2)

    response_3 = input(f"Enter the song title for the following lyrics: {question_1[1]}\n")
    if response_3.lower() == question_1[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_1[0]}. Your current score is {score}/30\n")

    #question 2
    question_2 = get_lyrics(response_2)
    response_5 = input(f"Enter the song title for the following lyrics: {question_2[1]}\n")
    if response_5.lower() == question_2[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_2[0]}. Your current score is {score}/30\n")

    #question 3
    question_3 = get_lyrics(response_2)
    response_7 = input(f"Enter the song title for the following lyrics: {question_3[1]}\n")
    if response_7.lower() == question_3[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_3[0]}. Your current score is {score}/30\n")

    return f"Your score is {score}/30."

In [34]:
root = tk.Tk()
root.geometry("610x760")

level2frame = tk.Frame(root, bg="#67D771")
level2frame.pack(fill="both", expand=True)

attempts = 0
points = 50
question_num = 0
max_questions = 5
used_lyrics = []
current_answer = ""
lyrics_label = None
entry = None
feedback_label = None
question_label = None
next_button = None
back_button = None


def show_welcome_screen():
    for widget in level2frame.winfo_children():
        widget.pack_forget()

    welcome_label = tk.Label(
        level2frame,
        text=(
            "🎵 Welcome to the Level 2: A lyric guessing game! 🎵\n\n"
            "You will be given a snippet of lyrics from songs by your chosen artist.\n"
            "- You start with 50 points.\n"
            "- If you guess wrong, you lose 5 points and receive a hint.\n"
            "- Guess wrong again or skip to the next question, you lose another 5 points.\n"
            "- The game ends after 5 questions.\n\n"
            "Type in an artist to begin!"
        ),
        font=("Helvetica", 14),
        wraplength=550,
        justify="left",
        bg="#67D771"
    )
    welcome_label.pack(pady=20)

    artist_label.pack(pady=10)
    artist_entry.pack(pady=5)
    artist_button.pack(pady=10)


def show_artist_input():
    for widget in level2frame.winfo_children():
        widget.pack_forget()

    artist_label.pack(pady=10)
    artist_entry.pack(pady=5)
    artist_button.pack(pady=10)

def get_lyrics_tk(artist):
    global back_button

    response = requests.get("https://api.genius.com/search", headers=HEADERS, params={"q": artist})
    data = response.json()

    if not data["response"]["hits"]:
        for widget in level2frame.winfo_children():
            widget.pack_forget()

        no_artist_label = tk.Label(level2frame, text="Artist not found. Please try again.", font=("Helvetica", 14), bg="#67D771")
        no_artist_label.pack(pady=10)

        back_button = tk.Button(level2frame, text="Back", font=("Helvetica", 14, "bold"), bg="white", command=show_artist_input)
        back_button.pack(pady=10)
        return None

    song = random.choice(data["response"]["hits"])["result"]
    title = song["title"]
    hint = song["title"][0]

    page = requests.get(song["url"])
    soup = BeautifulSoup(page.text, "html.parser")
    lyrics_divs = soup.find_all("div", class_=lambda x: x and "Lyrics__Container" in x)

    full_lyrics = ""
    for i in lyrics_divs:
        full_lyrics += i.text.strip()

    words = full_lyrics.split()
    if len(words) >= 10:
        stop = random.randint(10, len(words))
        start = stop - 10
        consecutive_words = words[start:stop]
        return [title, " ".join(consecutive_words), hint]
    else:
        for widget in level2frame.winfo_children():
            widget.pack_forget()

        no_lyrics_label = tk.Label(level2frame, text="Not enough lyrics. Please try a different artist.", font=("Helvetica", 14), bg="#67D771")
        no_lyrics_label.pack(pady=10)

        back_button = tk.Button(level2frame, text="Back", font=("Helvetica", 14, "bold"), bg="white", command=show_artist_input)
        back_button.pack(pady=10)
        return None


def start_game(artist):
    global question_num, used_lyrics, lyrics_label, entry, feedback_label, question_label, next_button, submit_button
    question_num = 0
    used_lyrics = []

    for widget in level2frame.winfo_children():
        widget.pack_forget()

    question_label = tk.Label(level2frame, text="", font=("Helvetica", 14, "bold"), bg="#67D771")
    lyrics_label = tk.Label(level2frame, text="", wraplength=500, bg="#67D771")
    entry = tk.Entry(level2frame, font=("Helvetica", 14, "bold"), bg="#67D771", bd=5)
    submit_button = tk.Button(level2frame, text="Submit", font=("Helvetica", 14, "bold"), bg="white", command=submit_answer)
    feedback_label = tk.Label(level2frame, text="", font=("Helvetica", 14, "bold"), bg="#67D771")
    next_button = tk.Button(level2frame, text="Next Question", font=("Helvetica", 14, "bold"), bg="white", command=lambda: next_question(artist))

    question_label.pack()
    lyrics_label.pack()
    entry.pack()
    submit_button.pack()
    feedback_label.pack()
    next_button.pack()

    next_button.config(state="disabled")

    next_question(artist)

def next_question(artist):
    global question_num, current_answer, used_lyrics, attempts, points

    if attempts == 1:
        points -= 5
        feedback_label.config(text=f"You skipped the second attempt. -5 points.\nCorrect answer was: {current_answer}", fg="red")

    attempts = 0 

    if question_num >= max_questions:
        question_label.config(text=f"Game Over! Final Score: {points} points")
        lyrics_label.config(text="")
        entry.pack_forget()
        next_button.pack_forget()
        submit_button.pack_forget()
        feedback_label.config(text="")
        return

    while True:
        lyrics = get_lyrics_tk(artist)
        if lyrics is None:
            feedback_label.config(text="Couldn't get lyrics. Try again.")
            return
        if lyrics not in used_lyrics:
            break

    used_lyrics.append(lyrics)
    current_answer = lyrics[0]
    question_num += 1

    question_label.config(text=f"Question {question_num}/{max_questions}: Guess the song")
    lyrics_label.config(text=f"{lyrics[1]}")
    feedback_label.config(text="")
    entry.delete(0, tk.END)

def submit_answer():
    global attempts, points

    guess = entry.get().strip()
    if guess.lower() == current_answer.lower():
        feedback_label.config(text=f"Correct! You have {points} points.", fg="green")
        next_button.config(state="normal")
    else:
        attempts += 1
        points -= 5
        if attempts == 1:
            feedback_label.config(
                text=f"Incorrect. Hint: The song starts with '{current_answer[0]}'. Try again. (-5 points)",
                fg="orange"
            )
        elif attempts == 2:
            feedback_label.config(
                text=f"Incorrect again. The correct answer was: {current_answer}. (-5 more points)",
                fg="red"
            )
            next_button.config(state="normal")

artist_button = tk.Button(level2frame, text="Submit", font=("Helvetica", 14, "bold"), bg="white",
                          command=lambda: start_game(artist_string.get()))

artist_string = tk.StringVar()
artist_entry = tk.Entry(level2frame, textvariable=artist_string, font=("Helvetica", 14), bg="#67D771", bd=5)
artist_label = tk.Label(level2frame, text="Enter artist name:", font=("Helvetica", 14, "bold"), bg="#67D771")

show_welcome_screen()

root.mainloop()